In [1]:
from trt_inferer import TRTInference
import tensorrt as trt
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.set_grad_enabled(False)
print(f"using torch {torch.__version__}")
print(f"using pyTensorRT {trt.__version__}")

using pyTensorRT 8.5.1.7
using torch 1.12.1+cu116
using pyTensorRT 8.5.1.7


# DUMMY model

In [2]:
HALF=True

In [3]:
class Clamp_fp16_bug(nn.Module):
    def __init__(self):
        super().__init__()
        self.pool = nn.AvgPool2d(kernel_size=(3,3))
    
    def forward(self, x):
        x = self.pool(x)# + 500
        x = torch.clamp(x, min=0., max=1000.)
        return x

In [4]:
inputs_t = torch.randn(1,3,400,400).cuda()
inputs_np = inputs_t.cpu().numpy()

In [5]:
model = Clamp_fp16_bug().cuda()

In [6]:

if HALF:
    model = model.half()
    inputs_t = inputs_t.half()

outputs = model(inputs_t)
outputs.max()

tensor(1.4951, device='cuda:0', dtype=torch.float16)

# Export to ONNX

In [7]:
torch.onnx.export(model,
                inputs_t,
                "bug.onnx",
                input_names=["inputs"],
                output_names=["outputs"],
                export_params=True,
                verbose=False,
                opset_version=16,
                do_constant_folding=True,
                training=torch.onnx.TrainingMode.EVAL,
                )

In [8]:
!/usr/src/tensorrt/bin/trtexec --onnx=bug.onnx --saveEngine=bug.engine --fp16

&&&& RUNNING TensorRT.trtexec [TensorRT v8501] # /usr/src/tensorrt/bin/trtexec --onnx=bug.onnx --saveEngine=bug.engine --fp16
[12/04/2022-14:20:54] [I] === Model Options ===
[12/04/2022-14:20:54] [I] Format: ONNX
[12/04/2022-14:20:54] [I] Model: bug.onnx
[12/04/2022-14:20:54] [I] Output:
[12/04/2022-14:20:54] [I] === Build Options ===
[12/04/2022-14:20:54] [I] Max batch: explicit batch
[12/04/2022-14:20:54] [I] Memory Pools: workspace: default, dlaSRAM: default, dlaLocalDRAM: default, dlaGlobalDRAM: default
[12/04/2022-14:20:54] [I] minTiming: 1
[12/04/2022-14:20:54] [I] avgTiming: 8
[12/04/2022-14:20:54] [I] Precision: FP32+FP16
[12/04/2022-14:20:54] [I] LayerPrecisions: 
[12/04/2022-14:20:54] [I] Calibration: 
[12/04/2022-14:20:54] [I] Refit: Disabled
[12/04/2022-14:20:54] [I] Sparsity: Disabled
[12/04/2022-14:20:54] [I] Safe mode: Disabled
[12/04/2022-14:20:54] [I] DirectIO mode: Disabled
[12/04/2022-14:20:54] [I] Restricted mode: Disabled
[12/04/2022-14:20:54] [I] Build only: Disab

[12/04/2022-14:21:09] [I] Average on 10 runs - GPU latency: 0.0148499 ms - Host latency: 0.133069 ms (enqueue 0.00969849 ms)
[12/04/2022-14:21:09] [I] Average on 10 runs - GPU latency: 0.0136047 ms - Host latency: 0.129077 ms (enqueue 0.00974731 ms)
[12/04/2022-14:21:09] [I] Average on 10 runs - GPU latency: 0.0128662 ms - Host latency: 0.129175 ms (enqueue 0.0100769 ms)
[12/04/2022-14:21:09] [I] Average on 10 runs - GPU latency: 0.0141296 ms - Host latency: 0.119763 ms (enqueue 0.00963745 ms)
[12/04/2022-14:21:09] [I] Average on 10 runs - GPU latency: 0.0148499 ms - Host latency: 0.128979 ms (enqueue 0.0109375 ms)
[12/04/2022-14:21:09] [I] Average on 10 runs - GPU latency: 0.0114563 ms - Host latency: 0.127991 ms (enqueue 0.0101318 ms)
[12/04/2022-14:21:09] [I] Average on 10 runs - GPU latency: 0.0158081 ms - Host latency: 0.142468 ms (enqueue 0.0112488 ms)
[12/04/2022-14:21:09] [I] Average on 10 runs - GPU latency: 0.0140442 ms - Host latency: 0.132562 ms (enqueue 0.0131165 ms)
[12/0

[12/04/2022-14:21:10] [I] 
[12/04/2022-14:21:10] [I] === Performance summary ===
[12/04/2022-14:21:10] [I] Throughput: 8120.9 qps
[12/04/2022-14:21:10] [I] Latency: min = 0.0925293 ms, max = 1.20801 ms, mean = 0.128448 ms, median = 0.130127 ms, percentile(90%) = 0.143311 ms, percentile(95%) = 0.148438 ms, percentile(99%) = 0.162598 ms
[12/04/2022-14:21:10] [I] Enqueue Time: min = 0.00463867 ms, max = 0.0561523 ms, mean = 0.0109925 ms, median = 0.00990295 ms, percentile(90%) = 0.0146484 ms, percentile(95%) = 0.0166016 ms, percentile(99%) = 0.0234375 ms
[12/04/2022-14:21:10] [I] H2D Latency: min = 0.0743408 ms, max = 1.15845 ms, mean = 0.0896455 ms, median = 0.0884399 ms, percentile(90%) = 0.0960388 ms, percentile(95%) = 0.098877 ms, percentile(99%) = 0.113525 ms
[12/04/2022-14:21:10] [I] GPU Compute Time: min = 0.00708008 ms, max = 0.0491943 ms, mean = 0.0139068 ms, median = 0.0133057 ms, percentile(90%) = 0.0205078 ms, percentile(95%) = 0.0217285 ms, percentile(99%) = 0.0263672 ms
[12/

# Check output via pyTensorRT

In [9]:
from trt_inferer import TRTInference
import tensorrt as trt

trt_model_path = "bug.engine"

inferer = TRTInference(trt_model_path)


inputs shape:[12/04/2022-14:21:10] [TRT] [W] CUDA lazy loading is not enabled. Enabling it can significantly reduce device memory usage. See `CUDA_MODULE_LOADING` in https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#env-vars
 (1, 3, 400, 400)
outputs shape: (1, 3, 133, 133)
input_dtype <class 'numpy.float16'>
output_dtype <class 'numpy.float16'>


In [11]:

trt_outp = inferer.infer(inputs_np.astype(inferer.input_dtype))
trt_outp#.max()


array([[[[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]],

        [[0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         ...,
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.],
         [0., 0., 0., ..., 0., 0., 0.]]]], dtype=float16)